<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [23]</a>'.</span>

In [1]:
import pandas as pd
import numpy as np
import pyodbc 
import time
import datetime
import sqlite3

In [2]:
import sys
import logging
path = "..\..\InSyncConnection\Code\clinical_log.txt"
logging.basicConfig(filename=path,
                    filemode='a',
                    format='%(asctime)s,%(msecs)d,%(name)s,%(levelname)s,%(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.DEBUG)
logger = logging.getLogger("NOMS-Survey Completion")

## DB Query

In [3]:
conn = sqlite3.connect('../../InSyncConnection/Database/InSyncClinical.db')
cursor = conn.cursor()

# # # printing all table names
# sql_query = """SELECT name FROM sqlite_master
#      WHERE type='table';"""

# cursor.execute(sql_query)
# print(cursor.fetchall())

### Encounter Query 

In [4]:
try:
    # Select all patients from EcounterSummary Table
    testPatients = '''
    SELECT
        PatientId
    FROM 
        emr_PatientDetails
    WHERE 
        LOWER(FirstName) LIKE '%test%'
        OR LOWER(LastName) LIKE '%test%'
        OR LOWER(FirstName) LIKE '%patient%'
        OR LOWER(LastName) LIKE '%patient%'
        or CAST(MRNNumber AS INTEGER) < 55
    '''
    sql=f'''
    SELECT 
        PatientId as PatientID,
        VisitDateTime as EncounterDate
    FROM 
        tblEncounterSummary
        LEFT JOIN tblENcounterType ON (tblEncounterSummary.EncounterTypeId=tblENcounterType.EncounterTypeId)
    WHERE 
        IsBillable = "TRUE"
        AND PatientID NOT IN ({testPatients})
    ORDER BY 
        PatientID
    '''
    encounter_df = pd.read_sql(sql, conn)
    encounter_df['EncounterDate'] = pd.to_datetime(encounter_df['EncounterDate'])
    
    #filter out encounters before 3/1 and keep the first encounter date
    encounter_filter = encounter_df['EncounterDate'] >= '2023-03-01'
    filtered_dates = encounter_df[encounter_filter].copy()
    filtered_dates.sort_values(by=['PatientID', 'EncounterDate'], inplace=True)
    encounter_df = filtered_dates.drop_duplicates(subset="PatientID", keep='first').copy()
    
    # get encounter time by hour
    encounter_df['Hour'] = encounter_df["EncounterDate"].dt.floor('h')
    
    # needed for filtering out transfer patients
    encounter_dictionary = dict(zip(encounter_df.PatientID, encounter_df.EncounterDate))
    
    logger.info(f"Successfully queried tblEncounterSummary.")
except Exception as e:
    logger.error(f"Failed to query tblEncounterSummary.", exc_info=True) 
    print(e)
    sys.exit(1)
encounter_df

C:\Users\YLen\AppData\Local\Temp\ipykernel_1160\2935963456.py:29: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  encounter_df['EncounterDate'] = pd.to_datetime(encounter_df['EncounterDate'])


,PatientID,EncounterDate,Hour
21,620288,2023-10-08 12:00:00,2023-10-08 12:00:00
122,620292,2023-07-05 17:00:00,2023-07-05 17:00:00
158,620293,2023-10-10 18:15:00,2023-10-10 18:00:00
255,620295,2023-03-05 13:00:00,2023-03-05 13:00:00
294,620298,2023-03-01 18:30:00,2023-03-01 18:00:00
...,...,...,...
178982,628026,2024-02-07 11:00:00,2024-02-07 11:00:00
178983,628035,2024-02-05 10:00:00,2024-02-05 10:00:00
178984,628041,2024-02-05 10:00:00,2024-02-05 10:00:00
178985,628060,2024-02-07 10:00:00,2024-02-07 10:00:00


### PatientDetails Query

In [5]:
try:
    # Select all patients
    sql= f'''
    SELECT 
        DISTINCT (PatientID) AS PatientID,
        FirstName,
        LastName,
        DOB,
        MRNNumber,
        PhoneNo AS Phone
    FROM 
        emr_PatientDetails
    WHERE
        PatientID NOT IN ({testPatients})
    ORDER BY 
        PatientID
    '''
    patient_details_df = pd.read_sql(sql, conn)
    
    # filter out MRN != XXX2
    MRN_filter = patient_details_df['MRNNumber'].map(lambda value: True if value[-1] == "2" else False)
    patient_details_df = patient_details_df[MRN_filter].copy()
    
    # filter out test patients
    test_filter = (patient_details_df['LastName'] != "Test") & (patient_details_df["FirstName"] != "Test")
    patient_details_df = patient_details_df[test_filter].copy()
    
    logger.info(f"Successfully queried emr_PatientDetails.")
except Exception as e:
    logger.error(f"Failed to query emr_PatientDetails.", exc_info=True) 
    print(e)
    sys.exit(1)

patient_details_df[patient_details_df['PatientID'].duplicated()]

,PatientID,FirstName,LastName,DOB,MRNNumber,Phone


### Discharged Query 

In [6]:
try:
    # Select all patients
    sql = f'''
    SELECT 
        PatientID,
        finalEncounterDate AS 'Final Encounter',
        is_ActiveInInSync,
        is_lastEncounterDischarge
    FROM 
        ptPatient_Activity
    WHERE
        PatientID NOT IN ({testPatients})
    '''
    disharge_df = pd.read_sql(sql, conn)
    
    # filter out active patients
    discharged_patients_mask = (disharge_df['is_ActiveInInSync'] == 0) | (disharge_df['is_lastEncounterDischarge'] == 1)
    discharged_patients_df = disharge_df[discharged_patients_mask].copy()
    
    # create discharge status
    discharged_patients_df['Status'] = "Discharged"
    
    discharged_patients_df.drop(columns=['is_ActiveInInSync','is_lastEncounterDischarge'],
                                axis=1,
                                inplace=True)
    discharged_patients_df['Final Encounter'] = pd.to_datetime(discharged_patients_df['Final Encounter'])
    
    logger.info(f"Successfully queried ptPatient_Activity.")
except Exception as e:
    logger.error(f"Failed to query ptPatient_Activity.", exc_info=True) 
    print(e)
    sys.exit(1)
discharged_patients_df

,PatientID,Final Encounter,Status
1,620290,2023-02-06 12:53:00,Discharged
2,620291,2022-07-07 12:27:00,Discharged
6,620296,2022-05-23 13:33:00,Discharged
8,620299,2022-03-14 23:09:00,Discharged
9,620300,2022-07-05 12:47:00,Discharged
...,...,...,...
5392,627554,2023-11-29 19:00:00,Discharged
5398,627561,2024-01-08 18:53:00,Discharged
5469,627657,2024-01-30 01:03:00,Discharged
5494,627690,2024-01-18 21:08:00,Discharged


### NOMS Query 

#### Baseline NOMS Taken

In [7]:
try:
    sql=f'''
    SELECT 
        PatientID,
        CreatedOn,
        StatusDesc AS 'Baseline NOMS'
    FROM 
        tblNOMS_AllAssessments
    WHERE 
        AssessmentKey LIKE '%Baseline%'
        AND (StatusDesc LIKE "%Entered%"
        OR StatusDesc LIKE "%Completed%")
        AND
        PatientID NOT IN ({testPatients})
    ORDER BY
        PatientID
    '''
    df = pd.read_sql(sql, conn)
    df['CreatedOn'] = pd.to_datetime(df['CreatedOn']).dt.date
    
    # filter before march
    march_mask = datetime.date(2023,3,1)
    baseline_df = df[df['CreatedOn'] >= march_mask].copy()
    
    # Prettify data for mergers later
    baseline_df['Baseline NOMS'] = baseline_df['Baseline NOMS'].map(lambda status: True if str(status) in ['Completed', "Entered Into SPARS"] else False)
    
    logger.info(f"Successfully queried tblNOMS_AllAssessments for Baselines.")
except Exception as e:
    logger.error(f"Failed to query tblNOMS_AllAssessments for Baselines.", exc_info=True) 
    print(e)
    sys.exit(1)

baseline_df

C:\Users\YLen\AppData\Local\Temp\ipykernel_1160\229098974.py:19: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['CreatedOn'] = pd.to_datetime(df['CreatedOn']).dt.date


,PatientID,CreatedOn,Baseline NOMS
0,620381,2023-07-31,True
1,620421,2023-05-03,True
2,620431,2023-04-25,True
3,620461,2023-05-11,True
4,620501,2023-05-30,True
...,...,...,...
347,627937,2024-01-24,True
348,627957,2024-01-29,True
349,627977,2024-01-29,True
350,627997,2024-02-05,True


#### Reassessment NOMS

In [8]:
try:
    sql=f'''
    SELECT 
        PatientID,
        CreatedOn
    FROM 
        tblNOMS_AllAssessments
    WHERE 
        AssessmentKey LIKE 'Reassessment%'
        AND (StatusDesc LIKE "%Entered%"
        OR StatusDesc LIKE "%Completed%")
        AND
        PatientID NOT IN ({testPatients})
    ORDER BY
        PatientID
    '''
    df = pd.read_sql(sql, conn)
    df['CreatedOn'] = pd.to_datetime(df['CreatedOn']).dt.date
    
    # filter before march
    march_mask = datetime.date(2023,3,1)
    reassessment_df = df[df['CreatedOn'] >= march_mask]
    
#     # find transfer patients
#     transfer_patients=[]
#     for patient in reassessment_df['PatientID']:
#         if False != encounter_dictionary.get(patient, False):
#             transfer_patients.append(patient)
            
#     # remove transfers if reassessment is before 6 months
#     earliest_reassessment_date = datetime.date(2023,8,1)
#     for patient in transfer_patients:
#         first_encounter = encounter_dictionary.get(patient)
#     #     incremented_first_encounter = first_encounter + np.timedelta64(6, 'M')
#         reassmessnent_date = df[df['PatientID'] == patient]['CreatedOn']
#         print(reassmessnent_date)
#         if (reassmessnent_date.values > earliest_reassessment_date) == False:
#             reassessment_df = reassessment_df[reassessment_df['PatientID'] != patient].copy()
    
    # Prettify data for megers later
    reassessment_df = reassessment_df[['PatientID']]
    reassessment_df['6 Month Reassessment NOMS'] = True
    
    logger.info(f"Successfully queried tblNOMS_AllAssessments for Reassessments.")
except Exception as e:
    logger.error(f"Failed to query tblNOMS_AllAssessments for Reassessments.", exc_info=True) 
    print(e)
    sys.exit(1)

#### Discharge NOMS

In [9]:
try:
    sql=f'''
    SELECT 
        DISTINCT (PatientID),
        CreatedOn AS 'Discharge Date',
        AssessmentKey AS 'Discharged'
    FROM 
        tblNOMS_AllAssessments
    WHERE 
        Discharged LIKE '%Discharge%'
        AND (StatusDesc LIKE "%Entered%"
        OR StatusDesc LIKE "%Completed%")
        AND
        PatientID NOT IN ({testPatients})
    ORDER BY
        PatientID
    '''
    df = pd.read_sql(sql, conn)
    df['Discharge Date'] = pd.to_datetime(df['Discharge Date']).dt.date
    
    # filter before march
    march_mask = datetime.date(2023,3,1)
    discharge_df = df[df['Discharge Date'] >= march_mask]
    
    
    # Prettify data for megers later
    discharge_df = discharge_df[['PatientID']]
    discharge_df.drop_duplicates(inplace=True)
    discharge_df['Discharge NOMS'] = True

    logger.info(f"Successfully queried tblNOMS_AllAssessments for Discharges.")
except Exception as e:
    logger.error(f"Failed to query tblNOMS_AllAssessments for Discharges.", exc_info=True) 
    print(e)
    sys.exit(1)
    
discharge_df

,PatientID,Discharge NOMS
0,620541,True
1,620671,True
2,620681,True
3,620721,True
4,620731,True
5,621201,True
7,621621,True
8,622191,True
9,623011,True
10,623041,True


#### NOMS Refusals

In [10]:
try:
    sql=f'''
    SELECT 
        PatientID,
        InterviewConductedNoID AS Reason
    FROM 
        tblNOMS_AllAssessments
    WHERE 
        AssessmentKey LIKE 'Base%'      AND
        InterviewConductedID LIKE '%0%' AND
        PatientID NOT IN ({testPatients})
    '''
    refused_df = pd.read_sql(sql, conn)
    refused_df['Consent'] = 'No'
    # fill in blank reasons with 'None Given'
    refused_df = refused_df.replace(r'^\s*$', 'None Given', regex=True)

    # Reorder columns
    refused_df = refused_df[['PatientID',
                             'Consent',
                             'Reason']]
    
    logger.info(f"Successfully queried tblNOMS_AllAssessments for Baseline Refusals.")
except Exception as e:
    logger.error(f"Failed to query tblNOMS_AllAssessments for Baseline Refusals.", exc_info=True) 
    print(e)
    sys.exit(1)
refused_df

,PatientID,Consent,Reason
0,621269,No,3 - Client Refused This Interview
1,620847,No,3 - Client Refused This Interview
2,620987,No,3 - Client Refused This Interview
3,622697,No,3 - Client Refused This Interview
4,624299,No,None Given
...,...,...,...
342,627977,No,4 - Client Was Not Reached For Interview
343,627957,No,3 - Client Refused This Interview
344,627997,No,3 - Client Refused This Interview
345,628007,No,4 - Client Was Not Reached For Interview


#### NOMS Consents

In [11]:
try:
    sql=f'''
    SELECT 
        PatientID
    FROM 
        tblNOMS_AllAssessments
    WHERE 
        AssessmentKey LIKE 'Base%'      AND
        InterviewConductedID LIKE '%1%' AND
        PatientID NOT IN ({testPatients})
    '''
    agreed_df = pd.read_sql(sql, conn)
    agreed_df
    # add Consent
    agreed_df["Consent"] = 'Yes'
    
    logger.info(f"Successfully queried tblNOMS_AllAssessments for Baseline Consents.")
except Exception as e:
    logger.error(f"Failed to query tblNOMS_AllAssessments for Baseline Consents.", exc_info=True) 
    print(e)
    sys.exit(1)
agreed_df

,PatientID,Consent
0,622516,Yes
1,622032,Yes
2,623392,Yes
3,620829,Yes
4,624114,Yes
...,...,...
315,623994,Yes
316,626905,Yes
317,627727,Yes
318,627707,Yes


#### Append Consent dfs 

In [12]:
consent_df = pd.concat([refused_df, agreed_df], ignore_index=True)

### Entered into Spars

#### Baselines

In [13]:
try:
    sql=f'''
    SELECT 
        PatientID,
        StatusDesc AS 'Baseline SPARS'
    FROM 
        tblNOMSAssessmentDetails
    WHERE
        PatientID NOT IN ({testPatients})
    ORDER BY
        PatientID
    '''
    baseline_SPARS = pd.read_sql(sql,conn)
        
    logger.info(f"Successfully queried tblNOMSAssessmentDetails for Baselines entered to SPARS.")
except Exception as e:
    logger.error(f"Failed to query tblNOMSAssessmentDetails for Baselines entered to SPARS.", exc_info=True) 
    print(e)
    sys.exit(1)

#### Reassessments

In [14]:
try:
    sql=f'''
    SELECT 
        PatientID,
        StatusDesc AS 'Reassessment SPARS'
    FROM 
        tblNOMSReAssessmentDetails
    WHERE
        PatientID NOT IN ({testPatients})
    ORDER BY
        PatientID
    '''
    reassessment_SPARS = pd.read_sql(sql,conn)
    
    logger.info(f"Successfully queried tblNOMSAssessmentDetails for Reassessments entered to SPARS.")
except Exception as e:
    logger.error(f"Failed to query tblNOMSAssessmentDetails for Reassessments entered to SPARS.", exc_info=True) 
    print(e)
    sys.exit(1)

#### Discharges 

In [15]:
try:
    sql=f'''
    SELECT 
        PatientID,
        StatusDesc AS 'Discharge SPARS'
    FROM 
        tblNOMSDischargeDetails
    WHERE
        PatientID NOT IN ({testPatients})
    ORDER BY
        PatientID
    '''
    discharge_SPARS = pd.read_sql(sql,conn)
    discharge_SPARS.drop_duplicates(inplace=True)    
    
    logger.info(f"Successfully queried tblNOMSAssessmentDetails for Discharges entered to SPARS.")
except Exception as e:
    logger.error(f"Failed to query tblNOMSAssessmentDetails for Discharges entered to SPARS.", exc_info=True) 
    print(e)
    sys.exit(1)
    

## Merges

### Merge patient_details and encounters 

In [16]:
try:
    full_data_collection_df = patient_details_df.merge(encounter_df,
                                                       on='PatientID',
                                                       how='left')
    # drop patients without encounters
    full_data_collection_df = full_data_collection_df[full_data_collection_df['EncounterDate'].notna()]
    
    logger.info(f"Successfully merged patient deatils with encounters.")
except Exception as e:
    logger.error(f"Failed to merge patient deatils with encounters.", exc_info=True) 
    print(e)
    sys.exit(1)

### Merge in Statuses

In [17]:
try:
    full_data_collection_df = full_data_collection_df.merge(discharged_patients_df,
                                                            on = 'PatientID',
                                                            how = 'left')
    # fill nan status
    full_data_collection_df['Status'].fillna('Active',inplace=True)
    
    logger.info(f"Successfully merged in discharged patients.")
except Exception as e:
    logger.error(f"Failed to merge in discharged patients.", exc_info=True) 
    print(e)
    sys.exit(1)   

### Merge in Baseline NOMS

In [18]:
try:
    full_data_collection_df = full_data_collection_df.merge(baseline_df,
                                                            on='PatientID',
                                                            how='left')
    full_data_collection_df['Baseline NOMS'].fillna(False,inplace=True)

    logger.info(f"Successfully merged in Baseline NOMS.")
except Exception as e:
    logger.error(f"Failed to merge in Baseline NOMS.", exc_info=True) 
    print(e)
    sys.exit(1)   

### Merge in Reassessment NOMS

In [19]:
try:
    full_data_collection_df = full_data_collection_df.merge(reassessment_df,
                                                            on='PatientID',
                                                            how='left')
    full_data_collection_df['6 Month Reassessment NOMS'].fillna(False,inplace=True)

    logger.info(f"Successfully merged in Reassessment NOMS.")
except Exception as e:
    logger.error(f"Failed to merge in Reassessment NOMS.", exc_info=True) 
    print(e)
    sys.exit(1) 

### Merge in Discharge NOMS 

In [20]:
try:
    full_data_collection_df = full_data_collection_df.merge(discharge_df,
                                                            on='PatientID',
                                                            how='left')
    full_data_collection_df['Discharge NOMS'].fillna(False,inplace=True)

    logger.info(f"Successfully merged in Discharge NOMS.")
except Exception as e:
    logger.error(f"Failed to merge in Discharge NOMS.", exc_info=True) 
    print(e)
    sys.exit(1) 

### Merge in SPARS Status

In [21]:
try: # add the baseline SPARS
    full_data_collection_df = full_data_collection_df.merge(baseline_SPARS,
                                                            on='PatientID',
                                                            how='left')
    logger.info(f"Successfully merged in Baseline SPARS.")
except Exception as e:
    logger.error(f"Failed to merge in Baseline SPARS.", exc_info=True) 
    print(e)
    sys.exit(1)
    

try: # add the reassessment SPARS
    full_data_collection_df = full_data_collection_df.merge(reassessment_SPARS,
                                                        on='PatientID',
                                                        how='left')
    logger.info(f"Successfully merged in Reassessment SPARS.")
except Exception as e:
    logger.error(f"Failed to merge in Reassessment SPARS.", exc_info=True) 
    print(e)
    sys.exit(1)


try: # add the discharge SPARS
    full_data_collection_df = full_data_collection_df.merge(discharge_SPARS,
                                                        on='PatientID',
                                                        how='left')
    logger.info(f"Successfully merged in Discharge SPARS.")
except Exception as e:
    logger.error(f"Failed to merge in Discharge SPARS.", exc_info=True) 
    print(e)
    sys.exit(1)

foo = full_data_collection_df.copy()

## Merge in New Notes

In [22]:
NoteSubTypeDict = {5: "First Attempt",
                    6: 'First Attempt',
                    7: "First Attempt"}
NoteSubTypeColumnDict = {5: 'Baseline SPARS',
                          6: "Reassessment SPARS",
                          7: "Discharge SPARS"}
try:
    sql=f'''
    SELECT 
        *
    FROM 
        tblPatientNotes
    WHERE
        NoteSubType IN (5, 6, 7)
    ORDER BY
        PatientID
    '''
    patientNotes = pd.read_sql(sql,conn)
    

    
    patientNotes['NoteAddedOn'] = pd.to_datetime(patientNotes['NoteAddedOn'])
    patientNotes.sort_values(by='NoteAddedOn', ascending=False)
    patientNotes['NoteSubTypeName'] = patientNotes['NoteSubType'].map(lambda subtypeID: NoteSubTypeDict[int(subtypeID)])
    patientNotes['NoteSubTypeColumn'] = patientNotes['NoteSubType'].map(lambda subtypeID: NoteSubTypeColumnDict[int(subtypeID)])
    patientNotesPivot = patientNotes.pivot(index=['PatientID'], columns = "NoteSubTypeColumn", values="NoteSubTypeName").reset_index()
    patientbaselineDict = patientNotesPivot.set_index('PatientID').to_dict()['Baseline SPARS']
    patientReassessmentDict = patientNotesPivot.set_index('PatientID').to_dict()['Reassessment SPARS']
    patientDischargeDict = patientNotesPivot.set_index('PatientID').to_dict()['Discharge SPARS']

    def checkforBaselineNotes(patientID, note):
        if note == "" or str(note) == 'nan':
            return patientbaselineDict.get(patientID, "")
        else:
            return note

    def checkforReassessmentNotes(patientID, note):
        if note == "" or str(note) == 'nan':
            return patientReassessmentDict.get(patientID, "")
        else:
            return note

    def checkforDischargeNotes(patientID, note):
        if note == "" or str(note) == 'nan':
            return patientDischargeDict.get(patientID, "")
        else:
            return note
    full_data_collection_df['Baseline SPARS'] = full_data_collection_df.apply(lambda row: checkforBaselineNotes(row['PatientID'], row['Baseline SPARS']), axis=1)
    full_data_collection_df['Reassessment SPARS'] = full_data_collection_df.apply(lambda row: checkforReassessmentNotes(row['PatientID'], row['Reassessment SPARS']), axis=1)
    full_data_collection_df['Discharge SPARS'] = full_data_collection_df.apply(lambda row: checkforDischargeNotes(row['PatientID'], row['Discharge SPARS']), axis=1)
#     logger.info(f"Successfully queried tblNotes and created columns.")
except Exception as e:
#     logger.error(f"Failed to query table notes and creat columns.", exc_info=True) 
    print(e)

## Prettify Data

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [23]:
full_data_collection_df = foo.copy()

try: # Format Names
    first_name = full_data_collection_df['FirstName'].map(lambda name: str(name).capitalize())
    last_name = full_data_collection_df['LastName'].map(lambda name: str(name).capitalize())
    full_data_collection_df['Name'] = last_name + ', ' + first_name
    full_data_collection_df.drop(columns=['FirstName','LastName'], inplace=True)
    
    logger.info(f"Successfully formated names.")
except Exception as e:
    logger.error(f"Failed to format names.", exc_info=True) 
    print(e)
    sys.exit(1)
    
try: # Baseline Date
    full_data_collection_df['Hour'] = pd.to_datetime(full_data_collection_df['Hour'])
    full_data_collection_df['Baseline Due Date'] = full_data_collection_df['Hour'] + np.timedelta64(30, 'D')
    full_data_collection_df['Baseline Due Date'] = full_data_collection_df['Baseline Due Date'].map(lambda num: num.strftime('%m-%d-%Y'))

    logger.info(f"Successfully added Baseline Due Date.")
except Exception as e:
    logger.error(f"Failed to add Baseline Due Date.", exc_info=True) 
    print(e)
    sys.exit(1)
    
try:    # Baseline Warning
    full_data_collection_df['Baseline Due Date'] = pd.to_datetime(full_data_collection_df['Baseline Due Date'])
    full_data_collection_df['Baseline Warning'] = (full_data_collection_df['Baseline Due Date'].dt.date <= datetime.date.today()) & (full_data_collection_df['Baseline NOMS'] == False)
    
    logger.info(f"Successfully added Baseline Warning.")
except Exception as e:
    logger.error(f"Failed to add Baseline Warning.", exc_info=True) 
    print(e)
    sys.exit(1)
    
try:    # 6 Month Reassessment Date
    full_data_collection_df['6 Month Reassessment Date'] = full_data_collection_df['Hour'] + np.timedelta64(6, 'M')
    full_data_collection_df['6 Month Reassessment Date'] = full_data_collection_df['6 Month Reassessment Date'].map(lambda num: num.strftime('%m-%d-%Y'))

    logger.info(f"Successfully added Reassessment Date.")
except Exception as e:
    logger.error(f"Failed to add Reassessment Date.", exc_info=True) 
    print(e)
    sys.exit(1)
    
try:    # 6 Month Warning
    full_data_collection_df['6 Month Reassessment Date'] = pd.to_datetime(full_data_collection_df['6 Month Reassessment Date'])
    full_data_collection_df['6 Month Reassessment Warning'] = full_data_collection_df['6 Month Reassessment Date'].dt.date - np.timedelta64(1, 'M') <= datetime.date.today()

    logger.info(f"Successfully added Reassessment Warning.")
except Exception as e:
    logger.error(f"Failed to add Reassessment Warning.", exc_info=True) 
    print(e)
    sys.exit(1)
    
try:    # add days since first encounter
    today = datetime.date.today()
    today = pd.to_datetime(today)
    def daysSinceFirstEncounter(row):
        date = row['EncounterDate']
        if type(date) == pd.Timestamp:
            return (today - date).days
        return np.nan
#     def daysTillBLineCompletion(row):
#         encounter_date = row['EncounterDate']
#         if isinstance(encounter_date, pd.Timestamp):
            
    def daysTillThirtyDaysAfter(row):
        encounter_date = row['EncounterDate']
        if isinstance(encounter_date, pd.Timestamp):
            # Calculate thirty days after the encounter date
            thirty_days_after = encounter_date + pd.DateOffset(days=30)
            days_till_thirty_days_after = (thirty_days_after - pd.Timestamp.today()).days
            return days_till_thirty_days_after
        return np.nan
    def daysTillSevenMonths(row):
        encounter_date = row['EncounterDate']
        if isinstance(encounter_date, pd.Timestamp):
            # Calculate seven months from the encounter date
            seven_months_later = encounter_date + pd.DateOffset(months=7)
            # Calculate the number of days between today and seven months later
            days_till_seven_months = (seven_months_later - pd.Timestamp.today()).days
            return days_till_seven_months
        return np.nan
    full_data_collection_df['Days Since Encounter'] = full_data_collection_df.apply(lambda row: daysSinceFirstEncounter(row), axis=1)  
    full_data_collection_df['Days Till Reassessment Date'] = full_data_collection_df.apply(lambda row: daysTillSevenMonths(row), axis=1)  
    full_data_collection_df['Days Till Baseline Date'] = full_data_collection_df.apply(lambda row: daysTillThirtyDaysAfter(row), axis=1)  
    logger.info(f"Successfully added Days Since First Encounter and days remaining till baseline/reassessment due dates.")
except Exception as e:
    logger.error(f"Failed to add Days Since First Encounter and days remaining till baseline/reassessment due dates.", exc_info=True) 
    print(e)
    sys.exit(1)

try:    # add days since final encounter
    def daysSinceDischarge(row):
        date = row['Final Encounter']
        if type(date) == pd.Timestamp:
            return (today - date).days
        return np.nan
    full_data_collection_df['Days Since Final Encounter'] = full_data_collection_df.apply(lambda row: daysSinceDischarge(row), axis=1)

    logger.info(f"Successfully added Days Since Final Encounter.")
except Exception as e:
    logger.error(f"Failed to add Days Since Final Encounter.", exc_info=True) 
    print(e)
    sys.exit(1)

Unit M is not supported. Only unambiguous timedelta values durations are supported. Allowed units are 'W', 'D', 'h', 'm', 's', 'ms', 'us', 'ns'


AttributeError: 'tuple' object has no attribute 'tb_frame'

In [ ]:
full_data_collection_df[full_data_collection_df['PatientID'].duplicated()]

In [ ]:
# # Reorder Columns
full_data_collection_df = full_data_collection_df[['PatientID',
                                                   'Name',
                                                   'DOB',
                                                   'Phone',
                                                   'MRNNumber',
                                                   'Status',
                                                   'EncounterDate',
                                                   'Hour',
                                                   'Days Since Encounter',
                                                   'Days Till Reassessment Date',
                                                   'Days Till Baseline Date',
                                                   'Baseline NOMS',
                                                   'Baseline Due Date',
                                                   'Baseline Warning',
                                                   'Baseline SPARS',
                                                   '6 Month Reassessment NOMS',
                                                   '6 Month Reassessment Date',
                                                   '6 Month Reassessment Warning',
                                                   'Reassessment SPARS',
                                                   'Final Encounter',
                                                   'Days Since Final Encounter',
                                                   'Discharge NOMS',
                                                   'Discharge SPARS']]
full_data_collection_df.drop_duplicates('PatientID', inplace=True)

## Push to DB

In [ ]:
# with pd.ExcelWriter(r"../data/NOMS Survey Completion.xlsx") as writer: 
#     full_data_collection_df.to_excel(writer, sheet_name="Patient Data",index = False)
#     consent_df.to_excel(writer, sheet_name="Consent Data",index = False)

In [ ]:
# ", ".join([item + " " + str(full_data_collection_df[item].dtype) for item in full_data_collection_df.columns])

### push full_data_collection

In [ ]:
table_name = "ptNOMS_Completion"
try:
    c = conn.cursor()
    c.execute(f'''CREATE TABLE IF NOT EXISTS {table_name} (PatientID INTEGER,
                                                           Name TEXT,
                                                           DOB TEXT,
                                                           Phone, TEXT
                                                           MRNNumber TEXT,
                                                           Status TEXT,
                                                           EncounterDate TEXT,
                                                           Hour TEXT,
                                                           DaysSinceEncounter INTEGER,
                                                           DaysTillBaseline INTEGER,
                                                           DaysTillReassessment INTEGER
                                                           BaselineNOMS TEXT,
                                                           BaselineDueDate TEXT,
                                                           BaselineWarning TEXT,
                                                           Baseline SPARS TEXT,
                                                           _6MonthReassessmentNOMS TEXT,
                                                           _6MonthReassessmentDate TEXT,
                                                           _6MonthReassessmentWarning TEXT,
                                                           ReassessmentSPARS TEXT,
                                                           FinalEncounter TEXT,
                                                           DaysSinceFinalEncounter INTEGER,
                                                           DischargeNOMS TEXT,
                                                           DischargeSPARS TEXT)''')
    conn.commit()
    logger.info(f"Successfully created {table_name}.")
except Exception as e:
    logger.error(f"Failed to create {table_name}.", exc_info=True) 
    print(e)
    
try:
    full_data_collection_df.to_sql(table_name, conn, if_exists='replace', index = False)
    logger.info(f"Successfully pushed data to {table_name}")
except Exception as e:
    logger.error(f"Failed to push data to {table_name}.", exc_info=True) 
    print(e)

### Push consent_df  

In [ ]:
# ", ".join([item + " " + str(consent_df[item].dtype) for item in consent_df.columns])

In [ ]:
table_name = "ptNOMS_Consent"
try:
    c = conn.cursor()
    c.execute(f'''CREATE TABLE IF NOT EXISTS {table_name} (PatientID INTEGER,
                                                           Consent TEXT,
                                                           Reason TEXT)''')
    conn.commit()
    logger.info(f"Successfully created {table_name}.")
except Exception as e:
    logger.error(f"Failed to create {table_name}.", exc_info=True) 
    print(e)
    
try:
    consent_df.to_sql(table_name, conn, if_exists='replace', index = False)
    logger.info(f"Successfully pushed data to {table_name}")
except Exception as e:
    logger.error(f"Failed to push data to {table_name}.", exc_info=True) 
    print(e)

In [ ]:
conn.close()